In [1]:
from utils import download_binary_file
import os
import shutil
import pandas as pd
import numpy as np
from scipy.io import mmread
import anndata
from anndata import AnnData
import scanpy as sc
from typing import List, Tuple

In [2]:
# Functions for downloading/reading files

def download_mcfarland_2020(output_path: str) -> None:
    """
    Download Mcfarland et al. 2020 data from the hosting URLs.

    Args:
    ----
        output_path: Output path to store the downloaded and unzipped
        directories.

    Returns
    -------
        None. File directories are downloaded and unzipped in output_path.
    """
    idasanutlin_url = "https://figshare.com/ndownloader/files/18716351"
    idasanutlin_output_filename = os.path.join(output_path, "idasanutlin.zip")

    download_binary_file(idasanutlin_url, idasanutlin_output_filename)
    idasanutlin_output_dir = idasanutlin_output_filename.replace(".zip", "")
    shutil.unpack_archive(idasanutlin_output_filename, idasanutlin_output_dir)

    dmso_url = "https://figshare.com/ndownloader/files/18716354"
    dmso_output_filename = os.path.join(output_path, "dmso.zip")

    download_binary_file(dmso_url, dmso_output_filename)
    dmso_output_dir = dmso_output_filename.replace(".zip", "")
    shutil.unpack_archive(dmso_output_filename, dmso_output_dir)


def _read_mixseq_df(directory: str) -> pd.DataFrame:
    data = mmread(os.path.join(directory, "matrix.mtx"))
    barcodes = pd.read_table(os.path.join(directory, "barcodes.tsv"), header=None)
    classifications = pd.read_csv(os.path.join(directory, "classifications.csv"))
    classifications["cell_line"] = np.array(
        [x.split("_")[0] for x in classifications.singlet_ID.values]
    )
    gene_names = pd.read_table(os.path.join(directory, "genes.tsv"), header=None)

    df = pd.DataFrame(
        data.toarray(),
        columns=barcodes.iloc[:, 0].values,
        index=gene_names.iloc[:, 0].values,
    )
    return df


def _get_cell_line_labels(directory: str) -> np.array:
    classifications = pd.read_csv(os.path.join(directory, "classifications.csv"))
    return classifications.singlet_ID.values


def _get_tp53_mutation_status(cell_line_labels: List[str]) -> np.array:
    # Taken from https://cancerdatascience.org/blog/posts/mix-seq/
    TP53_WT = [
        "LNCAPCLONEFGC_PROSTATE",
        "DKMG_CENTRAL_NERVOUS_SYSTEM",
        "NCIH226_LUNG",
        "RCC10RGB_KIDNEY",
        "SNU1079_BILIARY_TRACT",
        "CCFSTTG1_CENTRAL_NERVOUS_SYSTEM",
        "COV434_OVARY",
    ]

    TP53_mutation_status = [
        "Wild Type" if x in TP53_WT else "Mutation" for x in cell_line_labels
    ]
    return np.array(TP53_mutation_status)


def read_mcfarland_2020(file_directory: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Read the expression data for Mcfarland et al. 2020 in the given directory.

    Args:
    ----
        file_directory: Directory containing Mcfarland et al. 2020 data.

    Returns
    -------
        Two data frames of raw count expression data. The first contains
        single-cell gene expression count data from cancer cell lines exposed to
        idasanutlin with cell identification barcodes as column names and gene IDs as
        indices. The second contains count data with the same format from samples
        exposed to a control solution (DMSO).
    """
    idasanutlin_dir = os.path.join(
        file_directory, "idasanutlin", "Idasanutlin_24hr_expt1"
    )
    idasanutlin_df = _read_mixseq_df(idasanutlin_dir)

    dmso_dir = os.path.join(file_directory, "dmso", "DMSO_24hr_expt1")
    dmso_df = _read_mixseq_df(dmso_dir)

    return idasanutlin_df, dmso_df

In [3]:
download_path = "./"

idasanutlin_df, dmso_df = read_mcfarland_2020(download_path)
idasanutlin_df, dmso_df = idasanutlin_df.transpose(), dmso_df.transpose()

idasanutlin_adata = AnnData(idasanutlin_df)
idasanutlin_dir = os.path.join(
    download_path, "idasanutlin", "Idasanutlin_24hr_expt1"
)
idasanutlin_adata.obs["cell_line"] = _get_cell_line_labels(idasanutlin_dir)
idasanutlin_adata.obs["TP53_mutation_status"] = _get_tp53_mutation_status(
    idasanutlin_adata.obs["cell_line"]
)
idasanutlin_adata.obs["condition"] = np.repeat(
    "Idasanutlin", idasanutlin_adata.shape[0]
)

dmso_adata = AnnData(dmso_df)
dmso_dir = os.path.join(download_path, "dmso", "DMSO_24hr_expt1")
dmso_adata.obs["cell_line"] = _get_cell_line_labels(dmso_dir)
dmso_adata.obs["TP53_mutation_status"] = _get_tp53_mutation_status(
    dmso_adata.obs["cell_line"]
)
dmso_adata.obs["condition"] = np.repeat("DMSO", dmso_adata.shape[0])

adata = anndata.concat([idasanutlin_adata, dmso_adata])

/tmp/ipykernel_31619/762642982.py:6: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  idasanutlin_adata = AnnData(idasanutlin_df)
/tmp/ipykernel_31619/762642982.py:18: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  dmso_adata = AnnData(dmso_df)
/homes/gws/ewein/miniconda3/envs/contrastive-vi-env/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
# Filling in the standard metadata values

adata.obs['perturbation_name'] = adata.obs['condition']
adata.obs['perturbation_type'] = 'small molecule'
adata.obs['perturbation_value'] = '24'
adata.obs['perturbation_unit'] = 'hrs'